# <center> Arquitectura Transformers </center>

# Tema sobre el cual la artquitectura aprenderá: Propiedades Moleculares

<div style="display: flex; justify-content: center;">
  <img src="imagenes/potencial1.png" alt="Potencial Molecular" width="300" style="margin-right: 10px;">
  <img src="imagenes/potencial2.png" alt="Potencial Molecular" width="300" style="margin-right: 10px;">
  <img src="imagenes/potencial3.png" alt="Potencial Molecular" width="300">
</div>

##  Información sobre el tema 

En este trabajo, caracterizamos propiedades moleculares con la posición $\vec{r}$ y número atómico $z$ de los atómos que la componen (Variable $\bf X$)

## Cuantificación de esta información

Se cuenta con la estructura y propiedades de 130831 moleculas. La propiedad de interes (Variable $\bf Y$) es la energía interna a $0K$ $(U_0)$. Los valores de $\bf Y$ estan entre -19444.386 eV y -1101.487 eV

Base de datos: 

[torch_geometric.datasets.qm9](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.QM9.html)    

Articulos: 

[MoleculeNet: A Benchmark for Molecular Machine Learning](https://arxiv.org/abs/1703.00564) 

[Neural Message Passing for Quantum Chemistry](https://arxiv.org/abs/1704.01212)

## Entorno de ejecución

TorchMD-NET es un a arquitectura de equivariant Transformer (ET) la cual debe ser instalada en un ambiente virtual con los siguientes modulos.

**Dependencias:**
- h5py
- matplotlib
- nnpops==0.5
- pip
- pytorch==2.0.*
- pytorch_cluster==1.6.1
- pytorch_geometric==2.3.1
- pytorch_scatter==2.1.1
- pytorch_sparse==0.6.17
- pytorch-lightning==1.6.3
- torchmetrics==0.11.4
- tqdm

**Herramientas de desarrollo:**
- flake8
- pytest
- psutil
- ninja

Dependencias, herramientas de desarrollo y la arquitectura estan instaladas en un contenedor docker con nombre emmanuelzula/servicio:1.3 el cual puede ser descargado de [DockerHub](https://hub.docker.com/layers/emmanuelzula/servicio/1.3/images/sha256-6719a1f8ac3a8f45981b868b497c43f64df60da1d56a9a9122fecd9b522b6f39?context=repo).

## Importación y procesamiento de los datos en la artquitectura

TorchMD-NET tiene un modulo especializado en descargar y procesar el dataset QM9 el cual se encuentra en: 

torchmd-net/torchmdnet/datasets/qm9.py

## Exploración de los datos

La base de datos QM9 (Quantum Chemistry for Machine Learning). QM9 es una base de datos ampliamente utilizada en la química computacional y el aprendizaje automático para la predicción de propiedades moleculares. Contiene información sobre una amplia variedad de moléculas orgánicas pequeñas y sus propiedades calculadas mediante cálculos de química cuántica.

### Descarga y procesamiento del dataset para su analisis

In [1]:
import torch
from scripts.qm9 import QM9

# Crear una instancia de la clase QM9
dataset = QM9(root='data', transform=None, label='energy_U0')

dataset.download() # Descargar los datos

dataset.process()   # Procesar los datos

torch.save(dataset, '/workspace/clase_torchmd-net/data/dataset_qm9.pt')

/root/mambaforge/envs/torchmd-net/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data/raw/qm9_v3.zip
Using a pre-processed version of the dataset. Please install 'rdkit' to alternatively process the raw data.


La base de datos QM9 consta de 130831 elementos.

In [2]:
#Numero de muestras
print("El número de muestras es: " + str(len(dataset)))

El número de muestras es: 130831


Cada muestra consta de propiedades de moleculas simples, las cuales son: x, edge_index, edge_attr, y, pos, idx, name y z

In [3]:
dataset[0]  # Acceder al primer elemento del conjunto de datos

Data(x=[5, 11], edge_index=[2, 8], edge_attr=[8, 4], y=[1, 1], pos=[5, 3], idx=[1], name='gdb_1', z=[5])

Estas claves son:

1. `x`: Esta clave representa las características de los nodos en el grafo. En el contexto de la base de datos QM9, los nodos corresponden a átomos en una molécula. Por lo tanto, `x` contiene información sobre estos átomos. En este caso, hay 5 nodos (átomos) y cada uno tiene 11 características.

2. `edge_index`: Esta clave almacena la información de conectividad en el grafo. Indica cómo los nodos (átomos) están conectados entre sí mediante enlaces químicos. En este caso, hay 2 filas (representando pares de nodos conectados) y 8 columnas, lo que sugiere que hay 8 enlaces químicos en esta molécula.

3. `edge_attr`: Esta clave contiene atributos asociados a los bordes (enlaces químicos) del grafo. Hay 8 bordes en total, y cada borde tiene 4 características.

4. `y`: Esta clave representa el valor de la energía interna total (U_0) para la molécula. En este caso, parece haber una única etiqueta con valor 1, que corresponde a la energía total de la molécula.

5. `pos`: Esta clave contiene la posición tridimensional de los nodos en el espacio. En este caso, hay 5 nodos, y cada nodo tiene 3 coordenadas para su posición en el espacio tridimensional.

6. `idx`: Esta clave indica un índice único asociado a este elemento en particular. Puede ser útil para rastrear o identificar específicamente este elemento en la base de datos.

7. `name`: Esta clave generalmente almacena un nombre o identificador único para este elemento. En este caso, el nombre es "gdb_1".

8. `z`: Esta clave representa el número atómico de los átomos en la molécula. Hay 5 valores en total.

El modelo solamente utiliza los valores de "z", "pos" y "y", tomando "z" y "pos" como nuestra $\bf X$, en cambio tomanos "y" como nuestra $\bf Y$

## Contenidos de las entradas de nuestro interes

In [4]:
# Seleccionamos un elemento del dataset
elemento=dataset[0]

La llave "z" contiene los numeros atómicos de los atómos que componen a la molécula.

In [5]:

print(f"Key z:\n{elemento.z}")

Key x:
tensor([6, 1, 1, 1, 1])


La llave "pos" contiene las posiciones de los atómos que componen a la molécula en armstrong

In [6]:
print(f"Key pos:\n{elemento.pos}")

Key pos:
tensor([[-1.2700e-02,  1.0858e+00,  8.0000e-03],
        [ 2.2000e-03, -6.0000e-03,  2.0000e-03],
        [ 1.0117e+00,  1.4638e+00,  3.0000e-04],
        [-5.4080e-01,  1.4475e+00, -8.7660e-01],
        [-5.2380e-01,  1.4379e+00,  9.0640e-01]])


La llave "y" contiene la energía interna de la molécula una temperatura de $0K$ en eV

In [7]:
print(f"Key y:\n{elemento.y}")

Key y:
tensor([[-1101.4878]])


## Selección aleatoria de los datos

La función "train_val_test_split" en el archivo "torchmd-net/torchmdnet/utils.py" linea 54. Crea 3 arrays aleatorios

## Creación de los datasets learn (train y validation) y test

La función "make_splits" en el archivo "torchmd-net/torchmdnet/utils.py" linea 112. Utiliza los arrays para guardar un array con los indices seleccionados para los dataset train, val y test

La clase "DataModule" en el archivo "torchmd-net/torchmdnet/data.py" linea 90, 93 y 102. Crea los datasets train, val, y test

Se define "data" con la clase "DataModule" en el archivo "torchmd-net/torchmdnet/scripts/train.py" linea 128. Se ejecuta el codigo antes descrito

## Construcción de la arquitectura

La clase "TorchMD_ET" en el archivo "torchmd-net/torchmdnet/models/torchmd_et.py" linea 14. Define el modelo de transformer equivariante

La función "create_model" en el archivo "torchmd-net/torchmdnet/models/model.py" linea 15. Procesa y selecciona el modelo "TorchMD_ET"

La clase "LNNP" en el archivo "torchmd-net/torchmdnet/module.py" linea 12. Adapta el modelo creado para ser usado por pytorch_lightning

Se define "model" con la clase "LNNP" en el archivo "torchmd-net/torchmdnet/scripts/train.py" linea 136. Se ejecuta el codigo antes descrito

## Entrenamiento

Se define "trainer" en el archivo "torchmd-net/torchmdnet/scripts/train.py" linea 165. Compila el modelo construido

Se ejecuta la instrucción "trainer.fit(model, data)" en el achivo "torchmd-net/torchmdnet/scripts/train.py" linea 179. Inicia el entrenamiento.

## Analisis del entrenamiento

### Rutas y paquterias

In [8]:
# Ruta al archivo metrics 
path_metric_csv = "/workspace/tesis-potenciales-moleculares/output/metrics.csv"

path_splits_npy = "/workspace/tesis-potenciales-moleculares/output/splits.npz"

# Ruta al dataset que uso torchmd-net
path_torchmdnet_dataset = "/workspace/tesis-potenciales-moleculares/data/dataset_qm9.pt"

# Ruta al modelo entrenado
path_trained_model="/workspace/tesis-potenciales-moleculares/output/epoch=669-val_loss=0.0003-test_loss=0.0064.ckpt"

In [9]:
#Importar Modulos
import torch
import numpy as np
import pandas as pd
import os
from scripts.qm9 import QM9

### Muestra de la información

In [10]:
# Lee el archivo CSV en un DataFrame de pandas
metrics = pd.read_csv(path_metric_csv)

# Muestra las primeras filas del DataFrame para verificar la importación
print("Primeros datos")
print(metrics.head())

Primeros datos
   epoch        lr   train_loss  val_loss  step  test_loss
0    0.0  0.000023  1444.552734  8.349825   572        NaN
1    1.0  0.000046     2.451272  0.545728  1145        NaN
2    2.0  0.000069     0.280404  0.147569  1718        NaN
3    3.0  0.000092     0.115689  0.076925  2291        NaN
4    4.0  0.000115     0.068876  0.046653  2864        NaN


### Procesamiento de la información

In [11]:
# Asegurarse que los valores de epoch sean número enteros
metrics['epoch'] = metrics['epoch'].astype(int)

# Encontrar los índices donde test_loss está ausente
missing_indices = metrics[metrics['test_loss'].isnull()].index

# Eliminar las entradas en las demás columnas correspondientes a los índices de datos faltantes
metrics = metrics.drop(missing_indices, axis=0)

### Muestra de la información procesada

In [12]:
# Mostrar el DataFrame resultante
print("\nPrimeros datos eliminando filas con datos faltantes")
print(metrics.head())


Primeros datos eliminando filas con datos faltantes
    epoch        lr  train_loss  val_loss   step  test_loss
10     10  0.000252    0.098040  0.016314   6302   0.097029
20     20  0.000400    0.035254  0.006512  12032   0.059656
30     30  0.000400    0.023818  0.008792  17762   0.078178
40     40  0.000400    0.013053  0.072010  23492   0.262200
50     50  0.000400    0.002615  0.002128  29222   0.031002


In [13]:
# Mostrar el DataFrame resultante
print("\nUltimos datos eliminando filas con datos faltantes")
print(metrics.tail())


Ultimos datos eliminando filas con datos faltantes
     epoch            lr  train_loss  val_loss    step  test_loss
650    650  9.671406e-07    0.000004  0.000342  373022   0.006469
660    660  7.737125e-07    0.000004  0.000342  378752   0.006446
670    670  7.737125e-07    0.000004  0.000341  384482   0.006394
680    680  6.189700e-07    0.000005  0.000341  390212   0.006460
690    690  6.189700e-07    0.000005  0.000342  395942   0.006409


### Gráfica Learnig Rate en función de las Épocas

<div style="display: flex; justify-content: center;">
  <img src="imagenes/grafica4.png" alt="lr_vs_epoch_a" width="400" style="margin-right: 10px;">
  <img src="imagenes/grafica5.png" alt="lr_vs_epoch_b" width="400" style="margin-right: 10px;">
  <img src="imagenes/grafica6.png" alt="lr_vs_epoch_c" width="400">
</div>

### Gráfica MAE en función de las Épocas

<div style="display: flex; justify-content: center;">
  <img src="imagenes/grafica1.png" alt="MAE_vs_epoch_a" width="400" style="margin-right: 10px;">
  <img src="imagenes/grafica2.png" alt="MAE_vs_epoch_b" width="400" style="margin-right: 10px;">
  <img src="imagenes/grafica3.png" alt="MAE_vs_epoch_c" width="400">
</div>

## Resultados

Cargar archivos

In [14]:
# Cargar archivo del dataset que utiliza torchmd-net
torchmdnet_dataset = torch.load(path_torchmdnet_dataset)

# Cargar archivo splits
splits_data = np.load(path_splits_npy)

Almacenar datos para ser usados

In [15]:
# Acceder a la matriz específica dentro del archivo npz
splits = splits_data['idx_test'] 

# Indexar el dataset con los índices de splits
dataset_torchmdnet_test = torchmdnet_dataset[splits]

Cargar el modelo de predicción entrenado

In [16]:
from torchmdnet.models.model import load_model
model= load_model(path_trained_model, derivative=True)

Generar las inferencias de energia y fuerzas y guardarlas en un numpy array

In [17]:
if not os.path.exists('data/inferencias_test.dat'):
    with open('data/inferencias_test.dat', 'w') as file:
        for molecula in dataset_torchmdnet_test:
            z_sample = molecula.z
            pos_sample = molecula.pos
            n_energy_sample = molecula.y
            # Hacer la inferencia
            n_energy_inferred, n_forces_inferred = model(z_sample, pos_sample)
            # Transformar la inferencia
            n_energy_inferred = float(n_energy_inferred)
            n_energy_sample = float(n_energy_sample)        
            # Guardar los datos en el archivo
            file.write(f"{n_energy_sample} {n_energy_inferred}\n")
else:
    print("El archivo 'inferencias_test.dat' ya existe. No se ha realizado ninguna acción.")

El archivo 'inferencias_test.dat' ya existe. No se ha realizado ninguna acción.


Calcular metricas de error

In [18]:
from scripts.error import error

data = np.loadtxt('data/inferencias_test.dat')
energy_sample = data[:, 0]
energy_inferred = data[:, 1]

std_error=error("std","energy",energy_sample,energy_inferred)
mse_error=error("mse","energy",energy_sample,energy_inferred)
rmse_error=error("rmse","energy",energy_sample,energy_inferred)
mae_error=error("mae","energy",energy_sample,energy_inferred)
mae_error=mae_error*1000

print(f"\nLa desviación estandar es: {std_error}")
print(f"\nEl error cuadratico medio es: {mse_error}")
print(f"\nLa raiz del error cuadratico medio es: {rmse_error}")
print(f"\nEl error absoluto medio es: {mae_error:.3f} meV")


La desviación estandar es: 0.016072761509528743

El error cuadratico medio es: 0.0002583336625421887

La raiz del error cuadratico medio es: 0.016072761509528743

El error absoluto medio es: 6.368 meV


<p align="center">
  <img src="imagenes/tabla1.jpg" alt="Resultados_Articulo" width="900">
</p>

## Configuración de la arquitectura

La arquitectura TorchMD-NET requiere multiples parametros de configuración los cuales se agrupan de la siguiente manera

**Configuración general:**

- `--load-model`: Argumento para cargar un modelo desde un punto de control.
- `--conf` (`-c`): Especifica un archivo de configuración en formato YAML.
- `--num-epochs`: Número de épocas de entrenamiento.
- `--batch-size`: Tamaño del lote para el entrenamiento.
- `--inference-batch-size`: Tamaño del lote para la inferencia (validación y pruebas).
- `--lr`: Tasa de aprendizaje.
- `--lr-patience`: Paciencia para el ajuste de la tasa de aprendizaje.
- `--lr-metric`: Métrica utilizada para decidir cuándo reducir la tasa de aprendizaje.
- `--lr-min`: Tasa de aprendizaje mínima antes de detener el entrenamiento.
- `--lr-factor`: Factor para ajustar la tasa de aprendizaje.
- `--lr-warmup-steps`: Número de pasos de calentamiento para la tasa de aprendizaje.
- `--early-stopping-patience`: Paciencia para detener el entrenamiento si no mejora.
- `--reset-trainer`: Restablece las métricas de entrenamiento cuando se carga un punto de control.
- `--weight-decay`: Fuerza de la degradación de los pesos.
- `--ema-alpha-y`: Factor de influencia de las nuevas pérdidas en el promedio móvil exponencial de `y`.
- `--ema-alpha-neg-dy`: Factor de influencia de las nuevas pérdidas en el promedio móvil exponencial de `neg_dy`.
- `--ngpus`: Número de GPUs a utilizar.
- `--num-nodes`: Número de nodos.
- `--precision`: Precisión de punto flotante.
- `--log-dir` (`-l`): Directorio para los registros.
- `--splits`: Archivo NPZ con divisiones `idx_train`, `idx_val`, `idx_test`.
- `--train-size`: Porcentaje/número de muestras en el conjunto de entrenamiento.
- `--val-size`: Porcentaje/número de muestras en el conjunto de validación.
- `--test-size`: Porcentaje/número de muestras en el conjunto de pruebas.
- `--test-interval`: Intervalo de pruebas.
- `--save-interval`: Intervalo de guardado de modelos.
- `--seed`: Semilla aleatoria.
- `--num-workers`: Número de trabajadores para la obtención de datos.
- `--redirect`: Redirigir la salida estándar y de error al directorio de registro.
- `--gradient-clipping`: Norma de recorte del gradiente.

**Configuración de datos y modelo:**

- `--dataset`: Nombre del conjunto de datos Torch Geometric.
- `--dataset-root`: Directorio de almacenamiento de datos (no utilizado si el conjunto de datos es "CG").
- `--dataset-arg`: Argumentos adicionales para el conjunto de datos en formato JSON.
- `--coord-files`: Glob para archivos de coordenadas personalizados.
- `--embed-files`: Glob para archivos de incrustaciones personalizados.
- `--energy-files`: Glob para archivos de energía personalizados.
- `--force-files`: Glob para archivos de fuerza personalizados.
- `--y-weight`: Factor de ponderación para la etiqueta `y` en la función de pérdida.
- `--neg-dy-weight`: Factor de ponderación para `neg_dy` en la función de pérdida.

**Configuración de la arquitectura del modelo:**

- `--model`: Modelo a entrenar.
- `--output-model`: Tipo de modelo de salida.
- `--prior-model`: Modelo previo a utilizar.
- `--charge`: Indica si el modelo necesita una carga total.
- `--spin`: Indica si el modelo necesita un estado de espín.
- `--embedding-dimension`: Dimensión de la incrustación.
- `--num-layers`: Número de capas de interacción en el modelo.
- `--num-rbf`: Número de funciones de base radial en el modelo.
- `--activation`: Función de activación.
- `--rbf-type`: Tipo de expansión de distancia.
- `--trainable-rbf`: Si las funciones de expansión de distancia deben ser entrenables.
- `--neighbor-embedding`: Si se debe aplicar una incrustación de vecinos antes de las interacciones.
- `--aggr`: Operación de agregación para la salida del filtro CFConv.

**Configuración específica del Transformer:**

- `--distance-influence`: Donde se incluye la información de distancia en la atención.
- `--attn-activation`: Función de activación de atención.
- `--num-heads`: Número de cabezas de atención.

**Otros parámetros:**

- `--equivariance-invariance-group`: Grupo de equivarianza e invarianza de TensorNet.
- `--derivative`: Si es verdadero, toma la derivada de la predicción con respecto a las coordenadas.
- `--cutoff-lower`: Límite inferior en el modelo.
- `--cutoff-upper`: Límite superior en el modelo.
- `--atom-filter`: Suma solo sobre átomos con `Z > atom_filter`.
- `--max-z`: Número atómico máximo que cabe en la matriz de incrustación.
- `--max-num-neighbors`: Número máximo de vecinos a considerar en la red.
- `--standardize`: Si es verdadero, multiplica la predicción por la desviación estándar del conjunto de datos y agrega la media.
- `--reduce-op`: Operación de reducción para predicciones atómicas.
- `--wandb-use`: Si se usa Wandb (plataforma de seguimiento de experimentos).
- `--wandb-name`: Nombre para la ejecución de Wandb.
- `--wandb-project`: Proyecto Wandb al que se registran los experimentos.
- `--wandb-resume-from-id`: Reanudar un experimento Wandb a partir de un ID dado.
- `--tensorboard-use`: Si se usa TensorBoard (plataforma de seguimiento de experimentos de TensorFlow).

## Uso de la arquitectura

### Instalación de contenedor docker

Para instalar el contenedor docker es necesario tener instalado Docker en la computadora. Una vez instalado docker, ejecutamos el siguiente comando en una terminal en el directorio donde estemos trabajando

```bash
docker run -it --gpus all -p 9999:8888 -v $PWD:/workspace --name torchmd-net --shm-size 16G emmanuelzula/servicio:1.3 /bin/bash
```

Este comando se utiliza para ejecutar un contenedor Docker con ciertas configuraciones. Aquí está la lista:

1. **docker run**: Esto es el comando principal de Docker para ejecutar un contenedor.

2. **-it**: Estos son argumentos que se utilizan para indicar que se desea una terminal interactiva. Esto permite interactuar con el contenedor a través de la línea de comandos.

3. **--gpus all**: Indica que deseas asignar todos los recursos de GPU disponibles en el contenedor. Esto asume que tienes GPU y que has configurado Docker para admitir GPU.

4. **-p 9999:8888**: Esto mapea el puerto 8888 del contenedor al puerto 9999 de tu host local. Significa que si el contenedor ejecuta un servicio en el puerto 8888, podrás acceder a él desde tu navegador en `localhost:9999`.

5. **-v $PWD:/workspace**: Este argumento establece un volumen (mount) que vincula el directorio actual (`$PWD`) en tu host local con el directorio `/workspace` en el contenedor. Esto permite compartir archivos y datos entre tu sistema local y el contenedor.

6. **--name torchmd-net**: Asigna un nombre al contenedor, en este caso, "torchmd-net". Puedes usar este nombre para hacer referencia al contenedor en lugar de su identificador largo.

7. **--shm-size 16G**: Esto configura el tamaño de la memoria compartida (shared memory) dentro del contenedor en 16 gigabytes. Algunas aplicaciones pueden requerir más memoria compartida.

8. **emmanuelzula/servicio:1.3**: Es la imagen de Docker que se utilizará para crear el contenedor. En este caso, se está utilizando la imagen "emmanuelzula/servicio" con la etiqueta "1.3".

9. **/bin/bash**: Es el comando que se ejecutará dentro del contenedor. En este caso, se inicia un shell Bash dentro del contenedor, lo que te permite interactuar con él.

Una vez que termine de descargar e instalar el contenedor, por defecto estaremos dentro de él en la carpeta /workspace. Para salir del contenedor escribimos

```bash
exit
```

Para detener el contenedor escribimos en la terminal

```bash
docker stop torchmd-net
```

### Uso del contenedor

Primero debemos iniciar el contenedor con el siguiente comando en terminal

```bash
docker start torchmd-net
```

Una vez iniciado accedemos a él con el comando en terminal

```bash
docker exec -it torchmd-net /bin/bash/
```

Por defecto estaremos dentro de la carpeta del contenedor "/workspace" y con el ambiente virtual mamba "torchmd-net" activado. El el ambiente virtual se encuentara instaladas todas las dependencias, modulos y la arquitectura.

Adicionalmente se puede ejecutar jupyter notebook con el siguiente comando

```bash
jupyter notebook --ip='0.0.0.0' --port=8888 --no-browser --allow-root
```

Recordemos que el el puerto 8888 del contenedor se muestra en el puerto 9999 de la compuradora

### Ejecución de la arquitectura

La arquitectura requiere de: una carpeta de input, una carpeta de output y un script de arranque.

* En la carpeta input debera estar el archivo de configuración con extensión .yaml que debera contener toda la configuración de la arquitectura.
* En la carteta output se guardaran todos los resultados del entrenamiento.
* En el script de arranque deberan estar las instrucciónes de inicio y debera tener la estructura del siguiente ejemplo

```bash
CUDA_VISIBLE_DEVICES=1,2,3 torchmd-train --conf nuevos-entrenamientos/ET-QM9.yaml --log-dir qm9/
```

El cual configura los siguiente parametros de la forma:

- `CUDA_VISIBLE_DEVICES=1,2,3`: Este comando establece las GPU visibles para el proceso. En este caso, se configura para usar las GPU con identificadores 1, 2 y 3. Esto significa que el entrenamiento se realizará en estas tres GPU si están disponibles.

- `torchmd-train`: Este es el comando principal que se ejecuta. Parece ser una herramienta específica o un script de entrenamiento relacionado con TorchMD.

- `--conf nuevos-entrenamientos/ET-QM9.yaml`: Este es un argumento que proporciona la ruta al archivo de configuración YAML `ET-QM9.yaml` ubicado en el directorio `nuevos-entrenamientos`.

- `--log-dir qm9/`: Este argumento especifica el directorio donde se guardarán los registros del entrenamiento. En este caso, los registros se guardarán en un directorio llamado `qm9/`.

### Uso con nohup

Opcionalmente se ejecutar el script de forma independiente a la terminal, esto permite cerrar la terminal sin terminar el proceso iniciado por el script. Para hacerlo utilizamos nohup con el siguiente comando

```bash
CUDA_VISIBLE_DEVICES=1,2,3 nohup torchmd-net_qm9.sh > output.out &
```